<a href="https://colab.research.google.com/github/ObiAU/LoL-TensorFlow-Projects/blob/main/LoLWinPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Winrate base on Champion bans

### General Plan/Thoughts:

* will need to assign numerical values to the champions/download data set from kaggle.

* Will be a binary classification model (win or lose)

* Will compile with binary cross entropy and accuracy metric rather than mae/mse - I want to measure accuracy of winrate prediction to see if i can use it in realisitc settings.

* issue is how to train it/ is it necessary to take into account the user who bans the champion?

* Is it necessary/feasible to take into account the combination of 5 champion bans? That may be too complicated.

* It can be any given user.

* I will take 50,000 league ranked games as the general dataset. I will not do combination of champion bans but rather which champions were banned and how it affected the win or loss. This should output ~500,000 data units whilst taking into account champion multi-banning. (Maybe subset this) 

* x values will be what champions were banned.

* y values will be game outcome.

* x and y dimensions will not be equal. I need to figure out how to sort that out. 

* Let's say I test it by evaluating the model with an input of 'yasuo', then it needs to find all games where yasuo was banned and predict whether the player would win or lose.

## Training

* This is the crux of the difficulty (post-preprocessing). 

* Rather than thinking of x and y in terms of dimensions (layers should be able to sort that out for me).

* input layer will contain input shape argument to process this function

* output layer will obviously contain 1 neuron.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
# # Upload the file to colab the comment out (champion_info)

# from google.colab import files
# uploaded = files.upload()

In [ ]:
# Extract data from downloaded json file

import json

with open('champion_info.json', 'r') as json_file:
	json_load = json.load(json_file)

print(json_load), type(json_load)

In [ ]:
# Examine the data 
import io 

data = uploaded['champion_info.json'].decode("utf-8").split("\r\n")

for i in range(len(data)):
  data[i] = data[i].split(",")

print(data)


In [ ]:
# Assign full champion data

champion_info = uploaded['champion_info.json'].decode("utf-8")

champion_info = io.StringIO(champion_info)
champion_info = pd.read_csv(champion_info)
champion_info.info()

In [ ]:
# Upload new file(s) to colab and create function to generalise data
# extraction process then comment out

# data extractor
def load_data(file):
  return pd.read_csv(file)

In [ ]:
# # Upload files(games.csv)
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# # Upload files(champion_info2)
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# # Upload files(summoner_spell_info)
# from google.colab import files
# uploaded = files.upload()

In [ ]:
games = load_data('games.csv')

# use train_test_split to randomly split the data set into training and test data

from sklearn.model_selection import train_test_split

Gtrain, Gtest = train_test_split(games, test_size=0.2, random_state=5)
Gtrain.head(), Gtest.head()

(           gameId   creationTime  gameDuration  seasonId  winner  firstBlood  \
 50721  3329026671  1504462523534          1835         9       1           2   
 24254  3234057444  1498119534126          1828         9       2           1   
 6747   3286822044  1501604409968          1766         9       1           1   
 21010  3331169195  1504634601114          1636         9       1           1   
 1563   3329912774  1504546965196          1308         9       2           1   
 
        firstTower  firstInhibitor  firstBaron  firstDragon  ...  \
 50721           1               1           1            1  ...   
 24254           2               2           0            2  ...   
 6747            1               1           1            1  ...   
 21010           1               1           0            1  ...   
 1563            2               2           0            2  ...   
 
        t2_towerKills  t2_inhibitorKills  t2_baronKills  t2_dragonKills  \
 50721              1      

In [ ]:
Gtrain.head() # Examine the datas first 5 rows

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
50721,3329026671,1504462523534,1835,9,1,2,1,1,1,1,...,1,0,0,0,0,105,154,157,134,36
24254,3234057444,1498119534126,1828,9,2,1,2,2,0,2,...,10,2,0,4,0,236,90,154,134,498
6747,3286822044,1501604409968,1766,9,1,1,1,1,1,1,...,3,0,0,0,1,6,57,17,-1,53
21010,3331169195,1504634601114,1636,9,1,1,1,1,0,1,...,0,0,0,0,0,19,103,31,40,498
1563,3329912774,1504546965196,1308,9,2,1,2,2,0,2,...,9,2,0,1,0,105,63,67,75,51


In [ ]:
Gtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41192 entries, 50721 to 35683
Data columns (total 61 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gameId              41192 non-null  int64
 1   creationTime        41192 non-null  int64
 2   gameDuration        41192 non-null  int64
 3   seasonId            41192 non-null  int64
 4   winner              41192 non-null  int64
 5   firstBlood          41192 non-null  int64
 6   firstTower          41192 non-null  int64
 7   firstInhibitor      41192 non-null  int64
 8   firstBaron          41192 non-null  int64
 9   firstDragon         41192 non-null  int64
 10  firstRiftHerald     41192 non-null  int64
 11  t1_champ1id         41192 non-null  int64
 12  t1_champ1_sum1      41192 non-null  int64
 13  t1_champ1_sum2      41192 non-null  int64
 14  t1_champ2id         41192 non-null  int64
 15  t1_champ2_sum1      41192 non-null  int64
 16  t1_champ2_sum2      41192 non-null  

In [ ]:
# Check if the data set contains missing values
Gtrain.isnull().sum()

gameId          0
creationTime    0
gameDuration    0
seasonId        0
winner          0
               ..
t2_ban1         0
t2_ban2         0
t2_ban3         0
t2_ban4         0
t2_ban5         0
Length: 61, dtype: int64

In [ ]:
Gtrain['winner'].value_counts() # approximate 50%

1    20894
2    20298
Name: winner, dtype: int64

In [ ]:
Gtrain.groupby(['t2_ban1'])['seasonId'].value_counts().sort_values(ascending = False)

t2_ban1  seasonId
157      9           2661
31       9           2050
238      9           2036
40       9           1852
122      9           1805
                     ... 
82       9              9
13       9              7
72       9              6
14       9              6
77       9              4
Name: seasonId, Length: 139, dtype: int64

In [ ]:
# For our purpose we only need the t2 and t1 ban values
# Alongside win and loss binary labels

G_ban_train = Gtrain[['winner','t1_ban1', 't1_ban2', 't1_ban3'
, 't1_ban4', 't1_ban5','t2_ban1', 't2_ban2',
't2_ban3', 't2_ban4', 't2_ban5']].copy()


# Create new columns in the dataframe to store the lists
G_ban_train['Team_1_bans'] = 0
G_ban_train['Team_2_bans'] = 0

def List_Gen_Tool(df):
    df.reset_index(drop=True,inplace=True)
    df['Team_1_bans'] = df['Team_1_bans'].astype(object)
    df['Team_2_bans'] = df['Team_2_bans'].astype(object)
    for index in df.index:
        Team=df.loc[index][1:6].tolist()
        df.at[index,'Team_1_bans'] = Team
        Team2=df.loc[index][6:11].tolist()
        df.at[index,'Team_2_bans'] = Team2
    return df

In [ ]:
G_ban_train.head()

,winner,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5,Team_1_bans,Team_2_bans
50721,1,238,114,6,119,157,105,154,157,134,36,0,0
24254,2,154,114,92,157,245,236,90,154,134,498,0,0
6747,1,6,119,31,157,56,6,57,17,-1,53,0,0
21010,1,57,18,31,40,141,19,103,31,40,498,0,0
1563,2,40,21,28,55,23,105,63,67,75,51,0,0


In [ ]:
G_ban_train2 = List_Gen_Tool(G_ban_train)
G_ban_train2.head()

,winner,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5,Team_1_bans,Team_2_bans
0,1,238,114,6,119,157,105,154,157,134,36,"[238, 114, 6, 119, 157]","[105, 154, 157, 134, 36]"
1,2,154,114,92,157,245,236,90,154,134,498,"[154, 114, 92, 157, 245]","[236, 90, 154, 134, 498]"
2,1,6,119,31,157,56,6,57,17,-1,53,"[6, 119, 31, 157, 56]","[6, 57, 17, -1, 53]"
3,1,57,18,31,40,141,19,103,31,40,498,"[57, 18, 31, 40, 141]","[19, 103, 31, 40, 498]"
4,2,40,21,28,55,23,105,63,67,75,51,"[40, 21, 28, 55, 23]","[105, 63, 67, 75, 51]"


In [ ]:
# Now to drop the unnecessary rows and only have the final two
iGtrain = G_ban_train2[['winner', 'Team_1_bans', 'Team_2_bans']].copy()
iGtrain.head()

,winner,Team_1_bans,Team_2_bans
0,1,"[238, 114, 6, 119, 157]","[105, 154, 157, 134, 36]"
1,2,"[154, 114, 92, 157, 245]","[236, 90, 154, 134, 498]"
2,1,"[6, 119, 31, 157, 56]","[6, 57, 17, -1, 53]"
3,1,"[57, 18, 31, 40, 141]","[19, 103, 31, 40, 498]"
4,2,"[40, 21, 28, 55, 23]","[105, 63, 67, 75, 51]"


In [ ]:
# Repeat the process with the test data
G_ban_test = Gtest[['winner','t1_ban1', 't1_ban2', 't1_ban3'
, 't1_ban4', 't1_ban5','t2_ban1', 't2_ban2',
't2_ban3', 't2_ban4', 't2_ban5']].copy()


# Create new columns in the dataframe to store the lists
G_ban_test['Team_1_bans'] = 0
G_ban_test['Team_2_bans'] = 0

iGtest = List_Gen_Tool(G_ban_test)[['winner', 'Team_1_bans', 'Team_2_bans']].copy()

In [ ]:
# Check
iGtest, iGtrain

(       winner              Team_1_bans               Team_2_bans
 0           2    [69, 23, 86, 53, 122]  [141, 110, 57, 157, 202]
 1           1   [154, 31, 43, 11, 157]     [63, 67, 17, 45, 238]
 2           2  [117, 134, 31, 122, 29]    [24, 67, 51, 157, 107]
 3           2   [122, 154, 11, 18, 53]    [238, 75, 31, 154, 53]
 4           2   [31, 33, 238, 29, 157]     [92, 29, 18, 117, 40]
 ...       ...                      ...                       ...
 10293       2  [40, 117, 154, 119, 31]    [238, 53, 154, 6, 119]
 10294       1   [157, 133, 57, 31, 35]    [40, 18, 141, 90, 117]
 10295       2   [126, 55, 53, 119, 25]   [119, 67, 157, 141, 23]
 10296       1  [105, 75, 157, 18, 240]   [11, 238, 141, 119, 58]
 10297       2   [90, 114, 141, 23, 31]   [157, 119, 412, 31, 84]
 
 [10298 rows x 3 columns],
        winner               Team_1_bans               Team_2_bans
 0           1   [238, 114, 6, 119, 157]  [105, 154, 157, 134, 36]
 1           2  [154, 114, 92, 157, 245]  [23

In [ ]:
# We have test and train sets, time to begin building the model
# The test and train sets are in dataframe form
# We will need to extract the numerical values and lists

import numpy as np

xGtrain = np.array(iGtrain['Team_1_bans'].tolist())
xGtest = np.array(iGtest['Team_1_bans'].tolist())

yGtrain = np.array(iGtrain['winner'].tolist())
yGtest = np.array(iGtest['winner'].tolist())

# Check
yGtrain, yGtest, xGtrain, xGtest

xGtrain.shape

(41192, 5)

In [ ]:
from tensorflow.keras import layers

# Build the model
LoLWinPredictor = tf.keras.Sequential([
    layers.Dense(100, activation = "relu"),
    layers.Dense(10, activation = "relu"),
    layers.Dense(1, activation = "softmax")
])

LoLWinPredictor.compile(loss = "binary_crossentropy",
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics = ["accuracy"])

LoLWinPredictor.fit(xGtrain, yGtrain,
                    epochs = 5,
                    verbose = 1,
                    validation_data=(xGtest, yGtest))

Epoch 1/5
1288/1288 [==============================] - 7s 3ms/step - loss: -272363.8125 - accuracy: 0.5072 - val_loss: -1069437.3750 - val_accuracy: 0.5033
Epoch 2/5
1288/1288 [==============================] - 4s 3ms/step - loss: -3491432.7500 - accuracy: 0.5072 - val_loss: -6959896.5000 - val_accuracy: 0.5033
Epoch 3/5
1288/1288 [==============================] - 4s 3ms/step - loss: -12580538.0000 - accuracy: 0.5072 - val_loss: -19496898.0000 - val_accuracy: 0.5033
Epoch 4/5
1288/1288 [==============================] - 4s 3ms/step - loss: -28836200.0000 - accuracy: 0.5072 - val_loss: -39750612.0000 - val_accuracy: 0.5033
Epoch 5/5
1288/1288 [==============================] - 4s 3ms/step - loss: -53225276.0000 - accuracy: 0.5072 - val_loss: -68642792.0000 - val_accuracy: 0.5033
